In [ ]:
import pandas as pd
import numpy as np
from loaders import *

# TETRIS arch and mapping: with and without accumulation at DRAM

In [ ]:
show_config(ConfigRegistry.TETRIS_ARCH_DRAM_RED)
show_config(ConfigRegistry.TETRIS_COMPONENTS_DIR)
result = run_accelergy(ConfigRegistry.TETRIS_ARCH_DRAM_RED, ConfigRegistry.TETRIS_COMPONENTS_DIR)
print(result.ert)

In [ ]:
show_config(ConfigRegistry.TETRIS_ARCH_NO_DRAM_RED)
show_config(ConfigRegistry.TETRIS_COMPONENTS_DIR)
result = run_accelergy(ConfigRegistry.TETRIS_ARCH_NO_DRAM_RED, ConfigRegistry.TETRIS_COMPONENTS_DIR)
print(result.ert)

In [ ]:
def parse_stat(layer_stat):
    """
    Total number of compute in this layer
    Number of cycles in this layer
    Energy used in this layer in Joule
    Energy / Compute in picoJoule
    """
    lines = layer_stat.split("\n")
    compute = int(lines[-18].split("=")[-1].strip())
    cycle = int(lines[-23].split(":")[-1].strip())
    energy = float(lines[-22].split(":")[-1].strip()[:-3])
    return compute, cycle, energy * 10**(-6), energy * 10**(-6)  / compute * 10**(12)

### Example layer

In [ ]:
example_layer_stats, example_layer_mapping = run_timeloop_mapper(
    ConfigRegistry.TETRIS_ARCH_DRAM_RED, ConfigRegistry.TETRIS_COMPONENTS_DIR,
    ConfigRegistry.TETRIS_ARCH_CONSTRAINTS,
    ConfigRegistry.TETRIS_MAP_CONSTRAINTS,
    ConfigRegistry.VGG02_layer5, ConfigRegistry.DEFAULT_MAPPER_SETTING
)

In [ ]:
parse_stat(example_layer_stats)

### Squeeze Nets

In [ ]:
squeeze_layers = ConfigRegistry.SQUEEZE_NET_LAYERS
squeeze_stat = []
for i in range(len(squeeze_layers)):
    layer_stats, layer_mapping = run_timeloop_mapper(
        ConfigRegistry.TETRIS_ARCH_DRAM_RED, ConfigRegistry.TETRIS_COMPONENTS_DIR,
        ConfigRegistry.TETRIS_ARCH_CONSTRAINTS,
        ConfigRegistry.TETRIS_MAP_CONSTRAINTS,
        squeeze_layers[i+1], ConfigRegistry.DEFAULT_MAPPER_SETTING
    )
    squeeze_stat.append(layer_stats)

In [ ]:
for stat in squeeze_stat:
    compute, cycle, energy, energy_per_compute = parse_stat(stat)
    squeeze_total_compute += compute
    squeeze_total_cycle += cycle
    squeeze_total_energy += energy

print(f"Squeeze Net Total Compute:", squeeze_total_compute)
print(f"Squeeze Net Total Cycle:", squeeze_total_cycle)
print(f"Squeeze Net Total Energy:", squeeze_total_energy)
print(f"Squeeze Net Energy / Computer pJ / Compute", squeeze_total_energy / squeeze_total_compute * 10 ** (12))

### Mobile Nets

In [ ]:
mobile_layers = ConfigRegistry.MOBILE_NET_LAYERS
mobile_stat = []
for i in range(len(mobile_layers)):
    layer_stats, layer_mapping = run_timeloop_mapper(
        ConfigRegistry.TETRIS_ARCH_DRAM_RED, ConfigRegistry.TETRIS_COMPONENTS_DIR,
        ConfigRegistry.TETRIS_ARCH_CONSTRAINTS,
        ConfigRegistry.TETRIS_MAP_CONSTRAINTS,
        mobile_layers[i+1], ConfigRegistry.DEFAULT_MAPPER_SETTING
    )
    mobile_stat.append(layer_stats)

In [ ]:
mobile_total_compute = 0
mobile_total_cycle = 0
mobile_total_energy = 0

for stat in mobile_stat:
    compute, cycle, energy, energy_per_compute = parse_stat(stat)
    if compute is not None:
        mobile_total_compute += compute
        mobile_total_cycle += cycle
        mobile_total_energy += energy

print(f"Mobile Net Total Compute:", mobile_total_compute)
print(f"Mobile Net Total Cycle:", mobile_total_cycle)
print(f"Mobile Net Total Energy:", mobile_total_energy)
print(f"Mobile Net Energy / Computer pJ / Compute", mobile_total_energy / mobile_total_compute * 10 ** (12))

### Dense Net

In [ ]:
dense_layers = ConfigRegistry.DENSE_NET_LAYERS 
dense_stat = []
for i in range(len(dense_layers)):
    layer_stats, layer_mapping = run_timeloop_mapper(
        ConfigRegistry.TETRIS_ARCH_DRAM_RED, ConfigRegistry.TETRIS_COMPONENTS_DIR,
        ConfigRegistry.TETRIS_ARCH_CONSTRAINTS,
        ConfigRegistry.TETRIS_MAP_CONSTRAINTS,
        dense_layers[i+1], ConfigRegistry.DEFAULT_MAPPER_SETTING
    )
    dense_stat.append(layer_stats)

In [ ]:
dense_total_compute = 0
dense_total_cycle = 0
dense_total_energy = 0

for stat in dense_stat:
    compute, cycle, energy, energy_per_compute = parse_stat(stat)
    dense_total_compute += compute
    dense_total_cycle += cycle
    dense_total_energy += energy

print(f"Dense Net Total Compute:", dense_total_compute)
print(f"Dense Net Total Cycle:", dense_total_cycle)
print(f"Dense Net Total Energy:", dense_total_energy)
print(f"Dense Net Energy / Computer pJ / Compute", dense_total_energy / dense_total_compute * 10 ** (12))